In [15]:
from ctgan import CTGAN
from ctgan import load_demo
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import tensorflow as tf
print(torch.cuda.is_available())
import pandas as pd
import numpy as np

True


In [16]:
real_data = pd.read_csv("cicids2018.csv")

In [17]:
scaler = StandardScaler()
label = LabelEncoder()
# data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
#data[data.columns[:-1]] = np.array(data[data.columns[:-1]], dtype=float)
real_data.replace([np.inf, -np.inf], np.nan, inplace=True)
#data.iloc[:, -1] = label.fit_transform(data.iloc[:, -1])
real_data.dropna(inplace=True)
real_data.head(3).T

,0,1,2
Dst Port,80,80,80
Protocol,6,6,6
Flow Duration,5007793,8938396,52700
Tot Fwd Pkts,4,2,5
Tot Bwd Pkts,4,0,3
...,...,...,...
Idle Mean,0.0,8938396.0,0.0
Idle Std,0.0,0.0,0.0
Idle Max,0.0,8938396.0,0.0
Idle Min,0.0,8938396.0,0.0


In [18]:
def get_discrete_columns(dataframe, max_unique_values=10, convert_to_string=False):
    """
    Hàm để xác định các cột rời rạc (discrete), bao gồm các cột có số lượng giá trị unique ít hơn hoặc bằng max_unique_values.
    Ngoài ra, có thể chuyển đổi kiểu dữ liệu của các cột rời rạc sang string.
    
    - dataframe: DataFrame chứa dữ liệu
    - max_unique_values: Số lượng giá trị unique tối đa để coi là discrete (mặc định là 80)
    - convert_to_string: Nếu True, chuyển kiểu dữ liệu của các cột rời rạc sang string.
    
    Trả về:
    - discrete_columns: Danh sách tên các cột rời rạc.
    - discrete_indices: Danh sách chỉ số (index) của các cột rời rạc.
    """
    discrete_columns = []
    discrete_indices = []
    
    for col in dataframe.columns:
        unique_values = dataframe[col].nunique()
        print(f"Column '{col}' - Number of unique values: {unique_values}")
        # Chọn các cột có kiểu dữ liệu số và số lượng giá trị unique nhỏ hơn hoặc bằng max_unique_values
        if dataframe[col].dtype in ['float64', 'int64'] and unique_values <= max_unique_values:
            discrete_columns.append(col)
            discrete_indices.append(dataframe.columns.get_loc(col))
            
            # Nếu yêu cầu, chuyển đổi kiểu dữ liệu sang string
            if convert_to_string:
                dataframe[col] = dataframe[col].astype(str)
                print(f"Column '{col}' converted to string.")
    
    return discrete_columns, discrete_indices

# Ví dụ với dataset của bạn
discrete_columns, discrete_indices = get_discrete_columns(real_data)
discrete_columns.append("Label")
print("Discrete Columns:", discrete_columns)
print("Indices of Discrete Columns:", discrete_indices)

Column 'Dst Port' - Number of unique values: 37447
Column 'Protocol' - Number of unique values: 3
Column 'Flow Duration' - Number of unique values: 472240
Column 'Tot Fwd Pkts' - Number of unique values: 704
Column 'Tot Bwd Pkts' - Number of unique values: 942
Column 'TotLen Fwd Pkts' - Number of unique values: 6486
Column 'TotLen Bwd Pkts' - Number of unique values: 19272
Column 'Fwd Pkt Len Max' - Number of unique values: 1453
Column 'Fwd Pkt Len Min' - Number of unique values: 171
Column 'Fwd Pkt Len Mean' - Number of unique values: 31587
Column 'Fwd Pkt Len Std' - Number of unique values: 47062
Column 'Bwd Pkt Len Max' - Number of unique values: 1240
Column 'Bwd Pkt Len Min' - Number of unique values: 317
Column 'Bwd Pkt Len Mean' - Number of unique values: 40091
Column 'Bwd Pkt Len Std' - Number of unique values: 47467
Column 'Flow Byts/s' - Number of unique values: 526178
Column 'Flow Pkts/s' - Number of unique values: 534744
Column 'Flow IAT Mean' - Number of unique values: 4847

In [19]:
#real_data = pd.read_csv("iotid20.csv")

# Names of the columns that are discrete
#discrete_columns = [
    #'workclass',
    #'education',
    #'marital-status',
    #'occupation',
    #'relationship',
    #'race',
    ##'native-country',
    #'income'
#]

In [20]:
ctgan = CTGAN()

In [21]:
ctgan.fit(real_data, discrete_columns)
#ctgan.load_model("output\ctgan_model_epoch_40_1000.pth")

c:\Users\khang\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\autograd\graph.py:769: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np

def sample_data_by_class(ctgan, csv_path, label_column):
    """
    Sample synthetic data matching the class distribution in the provided dataset.

    Args:
        ctgan: Trained CTGAN model.
        csv_path (str): Path to the dataset (CSV file).
        label_column (str): Name of the column containing the labels.

    Returns:
        pandas.DataFrame: Synthetic data matching the class distribution.
    """
    # Đọc tập dữ liệu gốc
    df = pd.read_csv(csv_path)

    # Đếm số lượng mẫu của từng lớp
    label_counts = df[label_column].value_counts().to_dict()

    synthetic_data = []

    for label, count in label_counts.items():
        print(f"Sampling {count} rows for label '{label}'...")
        # Sử dụng CTGAN để sinh dữ liệu cho nhãn cụ thể
        sampled_data = ctgan.sample_by_label(n=count, label_column=label_column, label_value=label)
        synthetic_data.append(pd.DataFrame(sampled_data))  # Chuyển sang DataFrame

    # Kết hợp dữ liệu đã sinh
    synthetic_data = pd.concat(synthetic_data, ignore_index=True)
    return synthetic_data

csv_path = "cicids2018.csv"
label_column = "Target"
#synthetic_data = sample_data_by_class(ctgan, csv_path, label_column)
synthetic_data = ctgan.sample(len(real_data))


Sampling 143328 rows for label 'Mirai'...
Sampling 59362 rows for label 'DoS'...
Sampling 27530 rows for label 'Normal'...
Sampling 16274 rows for label 'Scan'...
Sampling 15044 rows for label 'MITM ARP Spoofing'...


In [ ]:
# Create synthetic data
#synthetic_data = ctgan.sample(250000)

In [ ]:
synthetic_data.to_csv("ctgan_10_2018.csv", index=False)